<a href="https://colab.research.google.com/github/Nayeong127/meotjin_yeongna/blob/main/220411_%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%8B%A4%EC%8A%B5_imdb%EC%84%B1%EB%8A%A590.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1st try**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb

In [ ]:
(X_train, y_train), (X_test, y_test) = imdb.load_data() #영화 리뷰 데이터를 로드.  #여기서 num_words를 사용하면 등장 빈도 순위로 몇 등까지의 단어를 사용할 것인지

print('훈련용 리뷰 개수 : {}'.format(len(X_train)))
print('테스트용 리뷰 개수 : {}'.format(len(X_test)))
num_classes = len(set(y_train))
print('카테고리 : {}'.format(num_classes))

17473536/17464789 [==============================] - 0s 0us/step
훈련용 리뷰 개수 : 25000
테스트용 리뷰 개수 : 25000
카테고리 : 2


In [ ]:
#X_train에 들어있는 숫자들이 각각 어떤 단어들을 나타내고 있는지 확인

word_to_index = imdb.get_word_index() #imdb.get_word_index()에 각 단어와 맵핑되는 정수가 저장! 근데 +3을 해야 실제 맵핑되는 정수
index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value+3] = key

1654784/1641221 [==============================] - 0s 0us/step


In [ ]:
#첫번째 훈련용 리뷰의 X_train[0]의 각 단어가 정수로 바뀌기 전에 어떤 단어들이었는지 확인
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index] = token

print(' '.join([index_to_word[index] for index in X_train[0]]))

<sos> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and shoul

In [ ]:
import re
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

vocab_size = 10000
max_len = 500

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
embedding_dim = 60 #이거 줄이고
hidden_units = 60 #이거 줄이고


#드롭아웃 0.2 하고
from keras.layers import Dropout
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(0.2)) 
model.add(GRU(hidden_units)) 
model.add(Dropout(0.2)) 
model.add(Dense(1, activation='sigmoid', kernel_regularizer='l2')) #엘투규제하고
model.add(Dropout(0.2)) 


es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) #얼스
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) 

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=32, validation_split=0.2) 

Epoch 1/15
625/625 [==============================] - ETA: 0s - loss: 1.9138 - acc: 0.7051
Epoch 1: val_acc improved from -inf to 0.85700, saving model to GRU_model.h5
625/625 [==============================] - 151s 239ms/step - loss: 1.9138 - acc: 0.7051 - val_loss: 0.3692 - val_acc: 0.8570
Epoch 2/15
625/625 [==============================] - ETA: 0s - loss: 1.8264 - acc: 0.7918
Epoch 2: val_acc improved from 0.85700 to 0.87780, saving model to GRU_model.h5
625/625 [==============================] - 148s 237ms/step - loss: 1.8264 - acc: 0.7918 - val_loss: 0.3451 - val_acc: 0.8778
Epoch 3/15
625/625 [==============================] - ETA: 0s - loss: 1.8359 - acc: 0.8129
Epoch 3: val_acc did not improve from 0.87780
625/625 [==============================] - 152s 244ms/step - loss: 1.8359 - acc: 0.8129 - val_loss: 0.3709 - val_acc: 0.8694
Epoch 4/15
625/625 [==============================] - ETA: 0s - loss: 1.7565 - acc: 0.8287
Epoch 4: val_acc did not improve from 0.87780
625/625 [===

In [ ]:
#  훈련 과정에서 검증 데이터의 정확도가 가장 높았을 때 저장된 모델인 'GRU_model.h5'를 로드

loaded_model = load_model('GRU_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 41s 53ms/step - loss: 0.3047 - acc: 0.8934

 테스트 정확도: 0.8934


In [ ]:
# 임의의 문장에 대해서 리뷰의 긍,부정을 예측하고자
# 모델에 넣기 전에 전처리를 해주어야!!

def sentiment_predict(new_sentence):  # 입력된 문장에 대해서 기본적인 전처리와 정수 인코딩, 패딩을 한 후에 모델의 입력으로 사용하여 예측값을 리턴!
  # 알파벳과 숫자를 제외하고 모두 제거 및 알파벳 소문자화
  new_sentence = re.sub('[^0-9a-zA-Z ]', '', new_sentence).lower()
  encoded = []

  # 띄어쓰기 단위 토큰화 후 정수 인코딩
  for word in new_sentence.split():
    try :
      # 단어 집합의 크기를 10,000으로 제한.
      if word_to_index[word] <= 10000:
        encoded.append(word_to_index[word]+3)
      else:
      # 10,000 이상의 숫자는 <unk> 토큰으로 변환.
        encoded.append(2)
    # 단어 집합에 없는 단어는 <unk> 토큰으로 변환.
    except KeyError:
      encoded.append(2)

  pad_sequence = pad_sequences([encoded], maxlen=max_len)
  score = float(loaded_model.predict(pad_sequence)) # 예측

  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))

In [ ]:
# 블랙팬서의 1점 리뷰 부정으로 예측?

neg1 = "This movie was just way too overrated. The fighting was not professional and in slow motion. I was expecting more from a 200 million budget movie. The little sister of T.Challa was just trying too hard to be funny. The story was really dumb as well. Don't watch this movie if you are going because others say its great unless you are a Black Panther fan or Marvels fan."

sentiment_predict(neg1)

97.17% 확률로 부정 리뷰입니다.


In [ ]:
#어벤져스의 10점 리뷰 긍정?

pos1 = " I was lucky enough to be included in the group to see the advanced screening in Melbourne on the 15th of April, 2012. And, firstly, I need to say a big thank-you to Disney and Marvel Studios. \
Now, the film... how can I even begin to explain how I feel about this film? It is, as the title of this review says a 'comic book triumph'. I went into the film with very, very high expectations and I was not disappointed. \
Seeing Joss Whedon's direction and envisioning of the film come to life on the big screen is perfect. The script is amazingly detailed and laced with sharp wit a humor. The special effects are literally mind-blowing and the action scenes are both hard-hitting and beautifully choreographed."

sentiment_predict(pos1)

90.41% 확률로 긍정 리뷰입니다.


ㅠ

# **2nd try**

In [ ]:
embedding_dim = 60 # 이거 줄인 거고
hidden_units = 60#  줄인 거고

import tensorflow as tf
initializer = tf.keras.initializers.GlorotNormal()

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units)) 
model.add(Dense(1, activation='sigmoid',kernel_initializer=initializer, kernel_regularizer='l2')) #가중치 초기화, 엘투규제

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5) #이번엔 검증 데이터 손실이 5회 증가하면 학습을 조기 종료
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) 

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc']) #아담
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=32, validation_split=0.15) #배치 사이즈 32

Epoch 1/15
665/665 [==============================] - ETA: 0s - loss: 0.5414 - acc: 0.7490
Epoch 1: val_acc improved from -inf to 0.76187, saving model to GRU_model.h5
665/665 [==============================] - 193s 286ms/step - loss: 0.5414 - acc: 0.7490 - val_loss: 0.5317 - val_acc: 0.7619
Epoch 2/15
665/665 [==============================] - ETA: 0s - loss: 0.5212 - acc: 0.7526
Epoch 2: val_acc did not improve from 0.76187
665/665 [==============================] - 196s 295ms/step - loss: 0.5212 - acc: 0.7526 - val_loss: 0.5461 - val_acc: 0.7237
Epoch 3/15
665/665 [==============================] - ETA: 0s - loss: 0.3611 - acc: 0.8571
Epoch 3: val_acc improved from 0.76187 to 0.87253, saving model to GRU_model.h5
665/665 [==============================] - 192s 289ms/step - loss: 0.3611 - acc: 0.8571 - val_loss: 0.3223 - val_acc: 0.8725
Epoch 4/15
665/665 [==============================] - ETA: 0s - loss: 0.2235 - acc: 0.9201
Epoch 4: val_acc improved from 0.87253 to 0.87360, saving 

In [ ]:
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 42s 53ms/step - loss: 0.3047 - acc: 0.8934

 테스트 정확도: 0.8934


In [ ]:
neg1 = "This movie was just way too overrated. The fighting was not professional and in slow motion. I was expecting more from a 200 million budget movie. The little sister of T.Challa was just trying too hard to be funny. The story was really dumb as well. Don't watch this movie if you are going because others say its great unless you are a Black Panther fan or Marvels fan."

pos1 = " I was lucky enough to be included in the group to see the advanced screening in Melbourne on the 15th of April, 2012. And, firstly, I need to say a big thank-you to Disney and Marvel Studios. \
Now, the film... how can I even begin to explain how I feel about this film? It is, as the title of this review says a 'comic book triumph'. I went into the film with very, very high expectations and I was not disappointed. \
Seeing Joss Whedon's direction and envisioning of the film come to life on the big screen is perfect. The script is amazingly detailed and laced with sharp wit a humor. The special effects are literally mind-blowing and the action scenes are both hard-hitting and beautifully choreographed."


In [ ]:
sentiment_predict(neg1)

97.17% 확률로 부정 리뷰입니다.


In [ ]:
sentiment_predict(pos1)

90.41% 확률로 긍정 리뷰입니다.


ㅠ

# **3.**

In [ ]:
# 단어 집합의 크기를 8000으로 제한

vocab_size = 80000
max_len = 500

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
embedding_dim = 60 # 이거 줄인 거고
hidden_units = 60#  줄인 거고

import tensorflow as tf
initializer = tf.keras.initializers.GlorotNormal()

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units)) 
model.add(Dense(1, activation='sigmoid',kernel_initializer=initializer, kernel_regularizer='l2')) #가중치 초기화, 엘투규제

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) 
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) 

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc']) #아담
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=32, validation_split=0.15) #배치 사이즈 32 밸리 줄임. 

Epoch 1/15
665/665 [==============================] - ETA: 0s - loss: 0.4649 - acc: 0.7832
Epoch 1: val_acc improved from -inf to 0.84400, saving model to GRU_model.h5
665/665 [==============================] - 223s 332ms/step - loss: 0.4649 - acc: 0.7832 - val_loss: 0.3781 - val_acc: 0.8440
Epoch 2/15
665/665 [==============================] - ETA: 0s - loss: 0.2724 - acc: 0.9012
Epoch 2: val_acc improved from 0.84400 to 0.86427, saving model to GRU_model.h5
665/665 [==============================] - 219s 329ms/step - loss: 0.2724 - acc: 0.9012 - val_loss: 0.3482 - val_acc: 0.8643
Epoch 3/15
665/665 [==============================] - ETA: 0s - loss: 0.1580 - acc: 0.9512
Epoch 3: val_acc improved from 0.86427 to 0.87467, saving model to GRU_model.h5
665/665 [==============================] - 222s 334ms/step - loss: 0.1580 - acc: 0.9512 - val_loss: 0.3638 - val_acc: 0.8747
Epoch 4/15
665/665 [==============================] - ETA: 0s - loss: 0.1059 - acc: 0.9715
Epoch 4: val_acc improve

얼리스타핑 내가 끊어버려야 할 것 같은데 ㅋ

In [ ]:
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))
#뭐여

InvalidArgumentError: ignored

In [ ]:
sentiment_predict(neg1)

97.17% 확률로 부정 리뷰입니다.


In [ ]:
sentiment_predict(pos1)

90.41% 확률로 긍정 리뷰입니다.


ㅠ

# **4.**

In [ ]:
# 단어 집합의 크기를 8000으로 제한
embedding_dim = 80 # 이거 줄인 거고
hidden_units = 80#  줄인 거고

import tensorflow as tf
initializer = tf.keras.initializers.GlorotNormal()

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units)) 
model.add(Dense(1, activation='sigmoid',kernel_initializer=initializer, kernel_regularizer='l2')) #가중치 초기화, 엘투규제

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) 
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) 

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc']) #아담
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=32, validation_split=0.25) #배치 사이즈 32 #밸리 키움

Epoch 1/15
586/586 [==============================] - ETA: 0s - loss: 0.5243 - acc: 0.7541
Epoch 1: val_acc improved from -inf to 0.83856, saving model to GRU_model.h5
586/586 [==============================] - 214s 362ms/step - loss: 0.5243 - acc: 0.7541 - val_loss: 0.3871 - val_acc: 0.8386
Epoch 2/15
586/586 [==============================] - ETA: 0s - loss: 0.2646 - acc: 0.9033
Epoch 2: val_acc improved from 0.83856 to 0.87152, saving model to GRU_model.h5
586/586 [==============================] - 210s 359ms/step - loss: 0.2646 - acc: 0.9033 - val_loss: 0.3288 - val_acc: 0.8715
Epoch 3/15
586/586 [==============================] - ETA: 0s - loss: 0.1419 - acc: 0.9569
Epoch 3: val_acc improved from 0.87152 to 0.87600, saving model to GRU_model.h5
586/586 [==============================] - 210s 359ms/step - loss: 0.1419 - acc: 0.9569 - val_loss: 0.3508 - val_acc: 0.8760
Epoch 4/15
586/586 [==============================] - ETA: 0s - loss: 0.0902 - acc: 0.9756
Epoch 4: val_acc did not

In [ ]:
loaded_model = load_model('GRU_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))
#bad.

782/782 [==============================] - 47s 60ms/step - loss: 0.3922 - acc: 0.8593

 테스트 정확도: 0.8593


In [ ]:
sentiment_predict(neg1)

97.37% 확률로 부정 리뷰입니다.


In [ ]:
sentiment_predict(pos1)

99.38% 확률로 긍정 리뷰입니다.


ㅠ

# **5.**

In [ ]:
# 단어 집합의 크기를 8000으로 제한
embedding_dim = 90 # 이거 줄인 거고
hidden_units = 90#  줄인 거고

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units)) 
model.add(Dense(1, activation='sigmoid',kernel_initializer=initializer, kernel_regularizer='l2')) #가중치 초기화, 엘투규제

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) 
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) 

model.compile(optimizer='adam', loss='mse', metrics=['acc']) #아담 #mse
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=32, validation_split=0.25) #배치 사이즈 32 #밸리 키움

Epoch 1/15
586/586 [==============================] - ETA: 0s - loss: 0.1631 - acc: 0.7692
Epoch 1: val_acc improved from -inf to 0.85648, saving model to GRU_model.h5
586/586 [==============================] - 242s 410ms/step - loss: 0.1631 - acc: 0.7692 - val_loss: 0.1127 - val_acc: 0.8565
Epoch 2/15
586/586 [==============================] - ETA: 0s - loss: 0.0790 - acc: 0.9048
Epoch 2: val_acc improved from 0.85648 to 0.86368, saving model to GRU_model.h5
586/586 [==============================] - 239s 409ms/step - loss: 0.0790 - acc: 0.9048 - val_loss: 0.1070 - val_acc: 0.8637
Epoch 3/15
586/586 [==============================] - ETA: 0s - loss: 0.0990 - acc: 0.8738
Epoch 3: val_acc did not improve from 0.86368
586/586 [==============================] - 238s 406ms/step - loss: 0.0990 - acc: 0.8738 - val_loss: 0.1099 - val_acc: 0.8608
Epoch 4/15
586/586 [==============================] - ETA: 0s - loss: 0.0407 - acc: 0.9581
Epoch 4: val_acc did not improve from 0.86368
586/586 [===

In [ ]:
loaded_model = load_model('GRU_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))
#뭐여

782/782 [==============================] - 54s 68ms/step - loss: 0.1186 - acc: 0.8537

 테스트 정확도: 0.8537


In [ ]:
sentiment_predict(neg1)

74.63% 확률로 부정 리뷰입니다.


In [ ]:
sentiment_predict(pos1)

98.02% 확률로 긍정 리뷰입니다.


ㅠ

# **6.**

In [ ]:
# 단어 집합의 크기를 8000으로 제한
embedding_dim = 100 
hidden_units = 128


import tensorflow as tf
initializer = tf.keras.initializers.GlorotNormal()

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units)) 
model.add(Dense(1, activation='sigmoid',kernel_initializer=initializer, kernel_regularizer='l2')) #가중치 초기화, 엘투규제

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) 
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) 

model.compile(optimizer='adam', loss='mse', metrics=['acc']) #아담 #mse
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=32, validation_split=0.25) #배치 사이즈 32 #밸리 키움

Epoch 1/15
586/586 [==============================] - ETA: 0s - loss: 0.1858 - acc: 0.7246
Epoch 1: val_acc improved from -inf to 0.82656, saving model to GRU_model.h5
586/586 [==============================] - 58s 88ms/step - loss: 0.1858 - acc: 0.7246 - val_loss: 0.1295 - val_acc: 0.8266
Epoch 2/15
586/586 [==============================] - ETA: 0s - loss: 0.1201 - acc: 0.8407
Epoch 2: val_acc improved from 0.82656 to 0.83456, saving model to GRU_model.h5
586/586 [==============================] - 50s 85ms/step - loss: 0.1201 - acc: 0.8407 - val_loss: 0.1258 - val_acc: 0.8346
Epoch 3/15
586/586 [==============================] - ETA: 0s - loss: 0.0925 - acc: 0.8839
Epoch 3: val_acc improved from 0.83456 to 0.84272, saving model to GRU_model.h5
586/586 [==============================] - 50s 86ms/step - loss: 0.0925 - acc: 0.8839 - val_loss: 0.1220 - val_acc: 0.8427
Epoch 4/15
586/586 [==============================] - ETA: 0s - loss: 0.0705 - acc: 0.9145
Epoch 4: val_acc improved from

In [ ]:
loaded_model = load_model('GRU_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))
#뭐여

782/782 [==============================] - 30s 38ms/step - loss: 0.1086 - acc: 0.8687

 테스트 정확도: 0.8687


In [ ]:
sentiment_predict(neg1)

97.33% 확률로 부정 리뷰입니다.


In [ ]:
sentiment_predict(pos1)

99.00% 확률로 긍정 리뷰입니다.


ㅡㅡ

# **7.**

In [ ]:
# 단어 집합의 크기를 8000으로 제한
embedding_dim = 60 #
hidden_units = 60#


import tensorflow as tf
initializer = tf.keras.initializers.GlorotNormal()

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units)) 
model.add(Dense(1, activation='sigmoid',kernel_initializer=initializer, kernel_regularizer='l2')) #가중치 초기화, 엘투규제

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) 
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) 

model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.5,name='binary_crossentropy'), metrics=['acc']) #아담 #binary_crossentropy with label smoothing
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=32, validation_split=0.25) #배치 사이즈 32 #밸리 키움

Epoch 1/15
586/586 [==============================] - ETA: 0s - loss: 0.6588 - acc: 0.7348
Epoch 1: val_acc improved from -inf to 0.82560, saving model to GRU_model.h5
586/586 [==============================] - 65s 106ms/step - loss: 0.6588 - acc: 0.7348 - val_loss: 0.6293 - val_acc: 0.8256
Epoch 2/15
586/586 [==============================] - ETA: 0s - loss: 0.6112 - acc: 0.8819
Epoch 2: val_acc improved from 0.82560 to 0.86304, saving model to GRU_model.h5
586/586 [==============================] - 65s 111ms/step - loss: 0.6112 - acc: 0.8819 - val_loss: 0.6164 - val_acc: 0.8630
Epoch 3/15
586/586 [==============================] - ETA: 0s - loss: 0.6020 - acc: 0.9073
Epoch 3: val_acc did not improve from 0.86304
586/586 [==============================] - 61s 104ms/step - loss: 0.6020 - acc: 0.9073 - val_loss: 0.6170 - val_acc: 0.8626
Epoch 4/15
586/586 [==============================] - ETA: 0s - loss: 0.5953 - acc: 0.9266
Epoch 4: val_acc improved from 0.86304 to 0.86864, saving mod

In [ ]:
loaded_model = load_model('GRU_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))
#뭐여

782/782 [==============================] - 28s 35ms/step - loss: 0.6222 - acc: 0.8627

 테스트 정확도: 0.8627


In [ ]:
sentiment_predict(neg1)

78.48% 확률로 부정 리뷰입니다.


In [ ]:
sentiment_predict(pos1)

77.90% 확률로 긍정 리뷰입니다.


# **8.**

In [ ]:
# 단어 집합의 크기를 8000으로 제한
embedding_dim = 60 
hidden_units = 60

#dropout!!!
from keras.layers import Dropout
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(0.2)) 
model.add(GRU(hidden_units)) 
model.add(Dropout(0.2)) 
model.add(Dense(1, activation='sigmoid', kernel_initializer=initializer,kernel_regularizer='l2')) ##가중치 초기화, 엘투규제
model.add(Dropout(0.2)) 

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) 
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) 

model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.5,name='binary_crossentropy'), metrics=['acc']) #아담 #binary_crossentropy with label smoothing
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=32, validation_split=0.25) #배치 사이즈 32 #밸리 키움

Epoch 1/15
586/586 [==============================] - ETA: 0s - loss: 2.0637 - acc: 0.6781
Epoch 1: val_acc improved from -inf to 0.79920, saving model to GRU_model.h5
586/586 [==============================] - 64s 106ms/step - loss: 2.0637 - acc: 0.6781 - val_loss: 0.6599 - val_acc: 0.7992
Epoch 2/15
586/586 [==============================] - ETA: 0s - loss: 2.0508 - acc: 0.7881
Epoch 2: val_acc did not improve from 0.79920
586/586 [==============================] - 61s 104ms/step - loss: 2.0508 - acc: 0.7881 - val_loss: 0.6747 - val_acc: 0.7750
Epoch 3/15
586/586 [==============================] - ETA: 0s - loss: 2.0054 - acc: 0.8291
Epoch 3: val_acc improved from 0.79920 to 0.82032, saving model to GRU_model.h5
586/586 [==============================] - 61s 104ms/step - loss: 2.0054 - acc: 0.8291 - val_loss: 0.6425 - val_acc: 0.8203
Epoch 4/15
586/586 [==============================] - ETA: 0s - loss: 2.0336 - acc: 0.8416
Epoch 4: val_acc improved from 0.82032 to 0.86624, saving mod

In [ ]:
loaded_model = load_model('GRU_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))
#뭐여

782/782 [==============================] - 28s 35ms/step - loss: 0.6304 - acc: 0.8695

 테스트 정확도: 0.8695


In [ ]:
sentiment_predict(neg1)

78.90% 확률로 부정 리뷰입니다.


In [ ]:
sentiment_predict(pos1)

61.29% 확률로 긍정 리뷰입니다.


# **9**

In [ ]:
# 단어 집합의 크기를 8000으로 제한
embedding_dim = 60 
hidden_units = 60

#dropout!!!
from keras.layers import Dropout
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(0.2)) 
model.add(GRU(hidden_units)) 
model.add(Dropout(0.2)) 
model.add(Dense(1, activation='sigmoid', kernel_initializer=initializer,kernel_regularizer='l2')) ##가중치 초기화, 엘투규제
model.add(Dropout(0.2)) 

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) 
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) 

model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.5,name='binary_crossentropy'), metrics=['acc']) #아담 #binary_crossentropy with label smoothing
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.25) #64

Epoch 1/15
293/293 [==============================] - ETA: 0s - loss: 2.0704 - acc: 0.6586
Epoch 1: val_acc improved from -inf to 0.79840, saving model to GRU_model.h5
293/293 [==============================] - 34s 109ms/step - loss: 2.0704 - acc: 0.6586 - val_loss: 0.6432 - val_acc: 0.7984
Epoch 2/15
293/293 [==============================] - ETA: 0s - loss: 2.0830 - acc: 0.7956
Epoch 2: val_acc improved from 0.79840 to 0.85984, saving model to GRU_model.h5
293/293 [==============================] - 31s 105ms/step - loss: 2.0830 - acc: 0.7956 - val_loss: 0.6284 - val_acc: 0.8598
Epoch 3/15
293/293 [==============================] - ETA: 0s - loss: 2.0594 - acc: 0.8106
Epoch 3: val_acc did not improve from 0.85984
293/293 [==============================] - 31s 105ms/step - loss: 2.0594 - acc: 0.8106 - val_loss: 0.6433 - val_acc: 0.8302
Epoch 4/15
293/293 [==============================] - ETA: 0s - loss: 2.0306 - acc: 0.8299
Epoch 4: val_acc improved from 0.85984 to 0.86640, saving mod

In [ ]:
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 27s 35ms/step - loss: 0.6304 - acc: 0.8695

 테스트 정확도: 0.8695


In [ ]:
neg1 = "This movie was just way too overrated. The fighting was not professional and in slow motion. I was expecting more from a 200 million budget movie. The little sister of T.Challa was just trying too hard to be funny. The story was really dumb as well. Don't watch this movie if you are going because others say its great unless you are a Black Panther fan or Marvels fan."

pos1 = " I was lucky enough to be included in the group to see the advanced screening in Melbourne on the 15th of April, 2012. And, firstly, I need to say a big thank-you to Disney and Marvel Studios. \
Now, the film... how can I even begin to explain how I feel about this film? It is, as the title of this review says a 'comic book triumph'. I went into the film with very, very high expectations and I was not disappointed. \
Seeing Joss Whedon's direction and envisioning of the film come to life on the big screen is perfect. The script is amazingly detailed and laced with sharp wit a humor. The special effects are literally mind-blowing and the action scenes are both hard-hitting and beautifully choreographed."


In [ ]:
sentiment_predict(neg1)

78.90% 확률로 부정 리뷰입니다.


In [ ]:
sentiment_predict(pos1)

61.29% 확률로 긍정 리뷰입니다.


저번에는 배치 작은 게 좋았는데 이번엔 큰 게 좋다네~하하~

# **9**

In [ ]:
# 단어 집합의 크기를 8000으로 제한
embedding_dim = 50 
hidden_units = 50

#dropout!!!
from keras.layers import Dropout
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(0.1)) 
model.add(GRU(hidden_units)) 
model.add(Dropout(0.1)) 
model.add(Dense(1, activation='sigmoid', kernel_initializer=initializer,kernel_regularizer='l2')) ##가중치 초기화, 엘투규제
model.add(Dropout(0.1)) 

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) 
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) 

model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.3,name='binary_crossentropy'), metrics=['acc']) #아담 #binary_crossentropy with label smoothing
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.25) #64

Epoch 1/15
293/293 [==============================] - ETA: 0s - loss: 1.3205 - acc: 0.7049
Epoch 1: val_acc improved from -inf to 0.84592, saving model to GRU_model.h5
293/293 [==============================] - 34s 107ms/step - loss: 1.3205 - acc: 0.7049 - val_loss: 0.5609 - val_acc: 0.8459
Epoch 2/15
293/293 [==============================] - ETA: 0s - loss: 1.2736 - acc: 0.8275
Epoch 2: val_acc improved from 0.84592 to 0.86320, saving model to GRU_model.h5
293/293 [==============================] - 31s 104ms/step - loss: 1.2736 - acc: 0.8275 - val_loss: 0.5453 - val_acc: 0.8632
Epoch 3/15
293/293 [==============================] - ETA: 0s - loss: 1.2422 - acc: 0.8659
Epoch 3: val_acc improved from 0.86320 to 0.86592, saving model to GRU_model.h5
293/293 [==============================] - 31s 104ms/step - loss: 1.2422 - acc: 0.8659 - val_loss: 0.5404 - val_acc: 0.8659
Epoch 4/15
293/293 [==============================] - ETA: 0s - loss: 1.1854 - acc: 0.8879
Epoch 4: val_acc did not im

In [ ]:
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 27s 35ms/step - loss: 0.6304 - acc: 0.8695

 테스트 정확도: 0.8695


In [ ]:
sentiment_predict(neg1)

78.90% 확률로 부정 리뷰입니다.


In [ ]:
sentiment_predict(pos1)

61.29% 확률로 긍정 리뷰입니다.


ㅠ

# FRESH START!!!!!!!!!!!다시 잘 생각해보자. 

##원래##

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb

In [ ]:
(X_train, y_train), (X_test, y_test) = imdb.load_data() #영화 리뷰 데이터를 로드.  #여기서 num_words를 사용하면 등장 빈도 순위로 몇 등까지의 단어를 사용할 것인지

print('훈련용 리뷰 개수 : {}'.format(len(X_train)))
print('테스트용 리뷰 개수 : {}'.format(len(X_test)))
num_classes = len(set(y_train))
print('카테고리 : {}'.format(num_classes))

17473536/17464789 [==============================] - 0s 0us/step
훈련용 리뷰 개수 : 25000
테스트용 리뷰 개수 : 25000
카테고리 : 2


In [ ]:
#X_train에 들어있는 숫자들이 각각 어떤 단어들을 나타내고 있는지 확인

word_to_index = imdb.get_word_index() #imdb.get_word_index()에 각 단어와 맵핑되는 정수가 저장! 근데 +3을 해야 실제 맵핑되는 정수
index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value+3] = key

1654784/1641221 [==============================] - 0s 0us/step


In [ ]:
#첫번째 훈련용 리뷰의 X_train[0]의 각 단어가 정수로 바뀌기 전에 어떤 단어들이었는지 확인
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index] = token

print(' '.join([index_to_word[index] for index in X_train[0]]))

<sos> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and shoul

In [ ]:
import re
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

vocab_size = 10000
max_len = 500

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [4]:
embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units)) 
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) 
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) 

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2) 

Epoch 1/15
313/313 [==============================] - ETA: 0s - loss: 0.4878 - acc: 0.7599
Epoch 1: val_acc improved from -inf to 0.85260, saving model to GRU_model.h5
313/313 [==============================] - 12s 31ms/step - loss: 0.4878 - acc: 0.7599 - val_loss: 0.3789 - val_acc: 0.8526
Epoch 2/15
311/313 [============================>.] - ETA: 0s - loss: 0.3125 - acc: 0.8737
Epoch 2: val_acc did not improve from 0.85260
313/313 [==============================] - 9s 30ms/step - loss: 0.3120 - acc: 0.8739 - val_loss: 0.4204 - val_acc: 0.8368
Epoch 3/15
312/313 [============================>.] - ETA: 0s - loss: 0.2434 - acc: 0.9087
Epoch 3: val_acc improved from 0.85260 to 0.88660, saving model to GRU_model.h5
313/313 [==============================] - 9s 30ms/step - loss: 0.2435 - acc: 0.9086 - val_loss: 0.2843 - val_acc: 0.8866
Epoch 4/15
311/313 [============================>.] - ETA: 0s - loss: 0.1882 - acc: 0.9304
Epoch 4: val_acc did not improve from 0.88660
313/313 [===========

In [5]:
#  훈련 과정에서 검증 데이터의 정확도가 가장 높았을 때 저장된 모델인 'GRU_model.h5'를 로드

loaded_model = load_model('GRU_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 7s 9ms/step - loss: 0.2849 - acc: 0.8921

 테스트 정확도: 0.8921


In [10]:
word_to_index = imdb.get_word_index() 
index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value+3] = key

# 임의의 문장에 대해서 리뷰의 긍,부정을 예측하고자
# 모델에 넣기 전에 전처리를 해주어야!!

def sentiment_predict(new_sentence):  # 입력된 문장에 대해서 기본적인 전처리와 정수 인코딩, 패딩을 한 후에 모델의 입력으로 사용하여 예측값을 리턴!
  # 알파벳과 숫자를 제외하고 모두 제거 및 알파벳 소문자화
  new_sentence = re.sub('[^0-9a-zA-Z ]', '', new_sentence).lower()
  encoded = []

  # 띄어쓰기 단위 토큰화 후 정수 인코딩
  for word in new_sentence.split():
    try :
      # 단어 집합의 크기를 10,000으로 제한.
      if word_to_index[word] <= 10000:
        encoded.append(word_to_index[word]+3)
      else:
      # 10,000 이상의 숫자는 <unk> 토큰으로 변환.
        encoded.append(2)
    # 단어 집합에 없는 단어는 <unk> 토큰으로 변환.
    except KeyError:
      encoded.append(2)

  pad_sequence = pad_sequences([encoded], maxlen=max_len)
  score = float(loaded_model.predict(pad_sequence)) # 예측

  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))

In [11]:
# 블랙팬서의 1점 리뷰 부정으로 예측?

neg1 = "This movie was just way too overrated. The fighting was not professional and in slow motion. I was expecting more from a 200 million budget movie. The little sister of T.Challa was just trying too hard to be funny. The story was really dumb as well. Don't watch this movie if you are going because others say its great unless you are a Black Panther fan or Marvels fan."

sentiment_predict(neg1)

98.94% 확률로 부정 리뷰입니다.


In [12]:
#어벤져스의 10점 리뷰 긍정?

pos1 = " I was lucky enough to be included in the group to see the advanced screening in Melbourne on the 15th of April, 2012. And, firstly, I need to say a big thank-you to Disney and Marvel Studios. \
Now, the film... how can I even begin to explain how I feel about this film? It is, as the title of this review says a 'comic book triumph'. I went into the film with very, very high expectations and I was not disappointed. \
Seeing Joss Whedon's direction and envisioning of the film come to life on the big screen is perfect. The script is amazingly detailed and laced with sharp wit a humor. The special effects are literally mind-blowing and the action scenes are both hard-hitting and beautifully choreographed."

sentiment_predict(pos1)

98.91% 확률로 긍정 리뷰입니다.


## **1st try**

In [17]:
embedding_dim = 60 #이거 줄이고
hidden_units = 60 #이거 줄이고


import tensorflow as tf
initializer = tf.keras.initializers.GlorotNormal()


model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units)) 
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) 
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) 

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2) 

Epoch 1/15
313/313 [==============================] - ETA: 0s - loss: 0.4597 - acc: 0.7794
Epoch 1: val_acc improved from -inf to 0.81740, saving model to GRU_model.h5
313/313 [==============================] - 8s 22ms/step - loss: 0.4597 - acc: 0.7794 - val_loss: 0.4125 - val_acc: 0.8174
Epoch 2/15
313/313 [==============================] - ETA: 0s - loss: 0.2942 - acc: 0.8822
Epoch 2: val_acc improved from 0.81740 to 0.87980, saving model to GRU_model.h5
313/313 [==============================] - 6s 20ms/step - loss: 0.2942 - acc: 0.8822 - val_loss: 0.2976 - val_acc: 0.8798
Epoch 3/15
313/313 [==============================] - ETA: 0s - loss: 0.2345 - acc: 0.9091
Epoch 3: val_acc improved from 0.87980 to 0.88800, saving model to GRU_model.h5
313/313 [==============================] - 6s 20ms/step - loss: 0.2345 - acc: 0.9091 - val_loss: 0.2747 - val_acc: 0.8880
Epoch 4/15
313/313 [==============================] - ETA: 0s - loss: 0.1960 - acc: 0.9273
Epoch 4: val_acc did not improve 

In [18]:
#  훈련 과정에서 검증 데이터의 정확도가 가장 높았을 때 저장된 모델인 'GRU_model.h5'를 로드

loaded_model = load_model('GRU_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 7s 8ms/step - loss: 0.2986 - acc: 0.8865

 테스트 정확도: 0.8865


In [19]:
sentiment_predict(neg1)

97.42% 확률로 부정 리뷰입니다.


In [20]:
sentiment_predict(pos1)

99.42% 확률로 긍정 리뷰입니다.


오히려 떨어짐~

##2
배치 줄임.

In [21]:
embedding_dim = 100
hidden_units = 128


import tensorflow as tf
initializer = tf.keras.initializers.GlorotNormal()


model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units)) 
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) 
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) 

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=32, validation_split=0.2)  ##배차사이즈

Epoch 1/15
625/625 [==============================] - ETA: 0s - loss: 0.4701 - acc: 0.7760
Epoch 1: val_acc improved from -inf to 0.84600, saving model to GRU_model.h5
625/625 [==============================] - 16s 23ms/step - loss: 0.4701 - acc: 0.7760 - val_loss: 0.3657 - val_acc: 0.8460
Epoch 2/15
625/625 [==============================] - ETA: 0s - loss: 0.2970 - acc: 0.8827
Epoch 2: val_acc improved from 0.84600 to 0.87980, saving model to GRU_model.h5
625/625 [==============================] - 14s 22ms/step - loss: 0.2970 - acc: 0.8827 - val_loss: 0.2845 - val_acc: 0.8798
Epoch 3/15
625/625 [==============================] - ETA: 0s - loss: 0.2034 - acc: 0.9219
Epoch 3: val_acc improved from 0.87980 to 0.89840, saving model to GRU_model.h5
625/625 [==============================] - 14s 22ms/step - loss: 0.2034 - acc: 0.9219 - val_loss: 0.2445 - val_acc: 0.8984
Epoch 4/15
624/625 [============================>.] - ETA: 0s - loss: 0.1548 - acc: 0.9439
Epoch 4: val_acc did not impro

In [22]:
#  훈련 과정에서 검증 데이터의 정확도가 가장 높았을 때 저장된 모델인 'GRU_model.h5'를 로드

loaded_model = load_model('GRU_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 7s 9ms/step - loss: 0.2905 - acc: 0.8970

 테스트 정확도: 0.8970


In [23]:
sentiment_predict(neg1)

99.65% 확률로 부정 리뷰입니다.


In [24]:
sentiment_predict(pos1)

99.46% 확률로 긍정 리뷰입니다.


배치가 작아졌을 때 좋았다. 그것은 그러면...!

##3

위백 선배님의 가르침... 레이블스무딩과 드랍아웃을 해봐라!!!
레이블스무딩
- label 0, 1을 잘못 라벨했을 수 있다는 가정!!!! 이떄 발생할 수 있는 잘못된 loss 의 영향을 줄이기 위하여 label 을 0또는 1이 아니라 smooth 하게 부여하는 것
  - ex) 0.2로 하면 계산해서 new_onehot_labels = 1 \* (1 — 0.2) + 0.2 / 2 = 1*(0.8) + 0.1
new_onehot_labels = 0.9 !! 일케 되면 0.9 정도면 잘 예측했다~


In [25]:
embedding_dim = 100
hidden_units = 128


import tensorflow as tf
initializer = tf.keras.initializers.GlorotNormal()


model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units)) 
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) 
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) 

model.compile(optimizer='rmsprop', loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.5), metrics=['acc']) ##label_smoothing
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=32, validation_split=0.2)  ##배차사이즈 

Epoch 1/15
625/625 [==============================] - ETA: 0s - loss: 0.6482 - acc: 0.7660
Epoch 1: val_acc improved from -inf to 0.82180, saving model to GRU_model.h5
625/625 [==============================] - 17s 25ms/step - loss: 0.6482 - acc: 0.7660 - val_loss: 0.6347 - val_acc: 0.8218
Epoch 2/15
625/625 [==============================] - ETA: 0s - loss: 0.6116 - acc: 0.8795
Epoch 2: val_acc improved from 0.82180 to 0.88180, saving model to GRU_model.h5
625/625 [==============================] - 14s 23ms/step - loss: 0.6116 - acc: 0.8795 - val_loss: 0.6087 - val_acc: 0.8818
Epoch 3/15
625/625 [==============================] - ETA: 0s - loss: 0.5971 - acc: 0.9153
Epoch 3: val_acc improved from 0.88180 to 0.88540, saving model to GRU_model.h5
625/625 [==============================] - 14s 23ms/step - loss: 0.5971 - acc: 0.9153 - val_loss: 0.6098 - val_acc: 0.8854
Epoch 4/15
625/625 [==============================] - ETA: 0s - loss: 0.5884 - acc: 0.9391
Epoch 4: val_acc improved from

In [26]:
#  훈련 과정에서 검증 데이터의 정확도가 가장 높았을 때 저장된 모델인 'GRU_model.h5'를 로드

loaded_model = load_model('GRU_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 7s 9ms/step - loss: 0.6094 - acc: 0.8848

 테스트 정확도: 0.8848


In [27]:
sentiment_predict(neg1)

76.69% 확률로 부정 리뷰입니다.


In [28]:
sentiment_predict(pos1)

73.78% 확률로 긍정 리뷰입니다.


##4
레이블스무딩 조정

In [30]:
embedding_dim = 100
hidden_units = 128


import tensorflow as tf
initializer = tf.keras.initializers.GlorotNormal()


model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units)) 
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) 
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) 

model.compile(optimizer='rmsprop', loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.2), metrics=['acc']) ##label_smoothing
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=32, validation_split=0.2)  ##배차사이즈 

Epoch 1/15
624/625 [============================>.] - ETA: 0s - loss: 0.5512 - acc: 0.7832
Epoch 1: val_acc improved from -inf to 0.86040, saving model to GRU_model.h5
625/625 [==============================] - 16s 23ms/step - loss: 0.5511 - acc: 0.7832 - val_loss: 0.4878 - val_acc: 0.8604
Epoch 2/15
624/625 [============================>.] - ETA: 0s - loss: 0.4559 - acc: 0.8922
Epoch 2: val_acc improved from 0.86040 to 0.89840, saving model to GRU_model.h5
625/625 [==============================] - 14s 22ms/step - loss: 0.4559 - acc: 0.8923 - val_loss: 0.4515 - val_acc: 0.8984
Epoch 3/15
624/625 [============================>.] - ETA: 0s - loss: 0.4172 - acc: 0.9242
Epoch 3: val_acc improved from 0.89840 to 0.90500, saving model to GRU_model.h5
625/625 [==============================] - 14s 22ms/step - loss: 0.4172 - acc: 0.9243 - val_loss: 0.4368 - val_acc: 0.9050
Epoch 4/15
624/625 [============================>.] - ETA: 0s - loss: 0.3942 - acc: 0.9449
Epoch 4: val_acc did not impro

In [31]:
#  훈련 과정에서 검증 데이터의 정확도가 가장 높았을 때 저장된 모델인 'GRU_model.h5'를 로드

loaded_model = load_model('GRU_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 7s 9ms/step - loss: 0.4401 - acc: 0.9022

 테스트 정확도: 0.9022


In [32]:
sentiment_predict(neg1)

85.07% 확률로 부정 리뷰입니다.


In [33]:
sentiment_predict(pos1)

91.19% 확률로 긍정 리뷰입니다.


ㄱㅇㄷ!!!!!!!!!!!!!!

##5
드랍아웃이랑 라벨스무딩 같이 했는데 구림. 

In [40]:
embedding_dim = 100
hidden_units = 128

#드롭아웃 0.2 
from keras.layers import Dropout
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(0.2)) 
model.add(GRU(hidden_units)) 
model.add(Dropout(0.2)) 
model.add(Dense(1, activation='sigmoid'))
model.add(Dropout(0.2)) 

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) 
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) 

model.compile(optimizer='rmsprop', loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.2), metrics=['acc']) ##label_smoothing
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=32, validation_split=0.2)  ##배차사이즈 

Epoch 1/15
625/625 [==============================] - ETA: 0s - loss: 2.0222 - acc: 0.7013
Epoch 1: val_acc improved from -inf to 0.83520, saving model to GRU_model.h5
625/625 [==============================] - 16s 23ms/step - loss: 2.0222 - acc: 0.7013 - val_loss: 0.5340 - val_acc: 0.8352
Epoch 2/15
625/625 [==============================] - ETA: 0s - loss: 1.9451 - acc: 0.7957
Epoch 2: val_acc improved from 0.83520 to 0.84280, saving model to GRU_model.h5
625/625 [==============================] - 14s 22ms/step - loss: 1.9451 - acc: 0.7957 - val_loss: 0.5286 - val_acc: 0.8428
Epoch 3/15
624/625 [============================>.] - ETA: 0s - loss: 1.9268 - acc: 0.8223
Epoch 3: val_acc improved from 0.84280 to 0.87980, saving model to GRU_model.h5
625/625 [==============================] - 14s 22ms/step - loss: 1.9252 - acc: 0.8225 - val_loss: 0.4962 - val_acc: 0.8798
Epoch 4/15
624/625 [============================>.] - ETA: 0s - loss: 1.9644 - acc: 0.8352
Epoch 4: val_acc improved from

In [41]:
#  훈련 과정에서 검증 데이터의 정확도가 가장 높았을 때 저장된 모델인 'GRU_model.h5'를 로드

loaded_model = load_model('GRU_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 7s 9ms/step - loss: 0.4824 - acc: 0.8938

 테스트 정확도: 0.8938


In [42]:
sentiment_predict(neg1)

91.55% 확률로 부정 리뷰입니다.


In [43]:
sentiment_predict(pos1)

69.15% 확률로 긍정 리뷰입니다.


얘도 높아지긴 함~!!

##6
- 드랍아웃

In [45]:
embedding_dim = 100
hidden_units = 128

#드롭아웃 0.2 
from keras.layers import Dropout
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(0.2)) 
model.add(GRU(hidden_units)) 
model.add(Dropout(0.2)) 
model.add(Dense(1, activation='sigmoid'))
model.add(Dropout(0.2)) 

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) 
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) 

model.compile(optimizer='rmsprop',  loss='binary_crossentropy', metrics=['acc']) 
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=32, validation_split=0.2)  ##배차사이즈 

Epoch 1/15
623/625 [============================>.] - ETA: 0s - loss: 2.2378 - acc: 0.6892
Epoch 1: val_acc improved from -inf to 0.80720, saving model to GRU_model.h5
625/625 [==============================] - 16s 23ms/step - loss: 2.2353 - acc: 0.6897 - val_loss: 0.4390 - val_acc: 0.8072
Epoch 2/15
625/625 [==============================] - ETA: 0s - loss: 1.8332 - acc: 0.7908
Epoch 2: val_acc improved from 0.80720 to 0.86860, saving model to GRU_model.h5
625/625 [==============================] - 14s 22ms/step - loss: 1.8332 - acc: 0.7908 - val_loss: 0.3423 - val_acc: 0.8686
Epoch 3/15
624/625 [============================>.] - ETA: 0s - loss: 1.8261 - acc: 0.8197
Epoch 3: val_acc improved from 0.86860 to 0.87800, saving model to GRU_model.h5
625/625 [==============================] - 14s 22ms/step - loss: 1.8272 - acc: 0.8197 - val_loss: 0.3214 - val_acc: 0.8780
Epoch 4/15
624/625 [============================>.] - ETA: 0s - loss: 1.7828 - acc: 0.8368
Epoch 4: val_acc improved from

In [46]:
#  훈련 과정에서 검증 데이터의 정확도가 가장 높았을 때 저장된 모델인 'GRU_model.h5'를 로드

loaded_model = load_model('GRU_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 7s 9ms/step - loss: 0.2530 - acc: 0.9014

 테스트 정확도: 0.9014


In [47]:
sentiment_predict(neg1)

97.03% 확률로 부정 리뷰입니다.


In [48]:
sentiment_predict(pos1)

91.18% 확률로 긍정 리뷰입니다.


ㄱㅇㄷ!!!!!!!


##7
- 드랍아웃 0.1

In [49]:
embedding_dim = 100
hidden_units = 128

#드롭아웃 0.2 
from keras.layers import Dropout
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(0.1)) 
model.add(GRU(hidden_units)) 
model.add(Dropout(0.1)) 
model.add(Dense(1, activation='sigmoid'))
model.add(Dropout(0.1)) 

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) 
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) 

model.compile(optimizer='rmsprop',  loss='binary_crossentropy', metrics=['acc']) 
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=32, validation_split=0.2)  ##배차사이즈 

Epoch 1/15
623/625 [============================>.] - ETA: 0s - loss: 1.2461 - acc: 0.7382
Epoch 1: val_acc improved from -inf to 0.86060, saving model to GRU_model.h5
625/625 [==============================] - 30s 26ms/step - loss: 1.2439 - acc: 0.7386 - val_loss: 0.3411 - val_acc: 0.8606
Epoch 2/15
623/625 [============================>.] - ETA: 0s - loss: 1.0358 - acc: 0.8412
Epoch 2: val_acc improved from 0.86060 to 0.89020, saving model to GRU_model.h5
625/625 [==============================] - 14s 23ms/step - loss: 1.0352 - acc: 0.8411 - val_loss: 0.3233 - val_acc: 0.8902
Epoch 3/15
625/625 [==============================] - ETA: 0s - loss: 1.0190 - acc: 0.8676
Epoch 3: val_acc improved from 0.89020 to 0.90180, saving model to GRU_model.h5
625/625 [==============================] - 14s 23ms/step - loss: 1.0190 - acc: 0.8676 - val_loss: 0.2666 - val_acc: 0.9018
Epoch 4/15
625/625 [==============================] - ETA: 0s - loss: 0.9714 - acc: 0.8867
Epoch 4: val_acc did not impro

In [50]:
#  훈련 과정에서 검증 데이터의 정확도가 가장 높았을 때 저장된 모델인 'GRU_model.h5'를 로드

loaded_model = load_model('GRU_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 7s 9ms/step - loss: 0.2785 - acc: 0.8923

 테스트 정확도: 0.8923


In [51]:
sentiment_predict(neg1)

97.24% 확률로 부정 리뷰입니다.


In [52]:
sentiment_predict(pos1)

90.59% 확률로 긍정 리뷰입니다.


크게 차이 안 나네.
